In [1]:
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from google.colab import drive


In [ ]:
data_dir = '/content/drive/MyDrive/video/'
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
def extract_frames(video_path, max_frames=20):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (160, 120))
        frames.append(frame)
    cap.release()
    while len(frames) < max_frames:
        frames.append(np.zeros((120, 160, 3), dtype=np.uint8))
    return np.array(frames)

In [ ]:
def load_data(data_dir, max_frames=20, num_classes=5):
    X = []
    y = []
    labels = os.listdir(data_dir)[:num_classes]
    label_dict = {label: idx for idx, label in enumerate(labels)}
    for label in labels:
        videos = os.listdir(os.path.join(data_dir, label))
        for video in videos:
            video_path = os.path.join(data_dir, label, video)
            frames = extract_frames(video_path, max_frames)
            X.append(frames)
            y.append(label_dict[label])
    X = np.array(X)
    y = to_categorical(y, num_classes=num_classes)
    return X, y, label_dict

X, y, label_dict = load_data(data_dir)

In [ ]:
model = Sequential()

# 3D CNN katmanları
model.add(Conv3D(32, (3, 3, 3), activation='relu', input_shape=(20, 120, 160, 3)))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Conv3D(64, (3, 3, 3), activation='relu'))
model.add(MaxPooling3D((2, 2, 2)))
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(64, activation='relu'))
model.add(Dense(len(label_dict), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 18, 118, 158, 32   2624      
                             )                                   
                                                                 
 max_pooling3d (MaxPooling3  (None, 9, 59, 79, 32)     0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 7, 57, 77, 64)     55360     
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 3, 28, 38, 64)     0         
 g3D)                                                            
                                                                 
 flatten (Flatten)           (None, 204288)            0         
                                                        

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/5
2/2 [==============================] - 85s 36s/step - loss: 1223.9791 - accuracy: 0.2200 - val_loss: 1971.1389 - val_accuracy: 0.1538
Epoch 2/5
2/2 [==============================] - 78s 32s/step - loss: 1725.9056 - accuracy: 0.2400 - val_loss: 143.0556 - val_accuracy: 0.1538
Epoch 3/5
2/2 [==============================] - 78s 32s/step - loss: 87.5024 - accuracy: 0.3200 - val_loss: 8.2967 - val_accuracy: 0.4615
Epoch 4/5
2/2 [==============================] - 82s 35s/step - loss: 12.1775 - accuracy: 0.2800 - val_loss: 4.8290 - val_accuracy: 0.3077
Epoch 5/5
1/1 [==============================] - 7s 7s/step - loss: 1.4710 - accuracy: 0.3125
Test Accuracy: 31.25%


In [ ]:
feature_extractor = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)

def extract_features(video_path):
    frames = extract_frames(video_path, max_frames=20)
    frames = np.expand_dims(frames, axis=0)  # Modelin giriş boyutuna uygun hale getirme
    features = feature_extractor.predict(frames)
    return features

# Örnek video için özellik çıkarımı
example_video_path = '/content/drive/MyDrive/video/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi'
features = extract_features(example_video_path)
print(features.shape)  # (1, 64)
# Özellik vektörünü yazdırma (formatlı)
print("Extracted features (formatted):")
print(features[0])

1/1 [==============================] - 0s 420ms/step
(1, 64)
Extracted features (formatted):
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.41926121e-03
 2.50633084e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.74448423e-03 0.00000000e+00 0.00000000e+00
 4.51825443e-04 0.00000000e+00 0.00000000e+00 1.19723336e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.08064571e-03 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.32772173e-04 8.33252561e-05
 6.44692627e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000